In [52]:
gpu=3
CUDA_VISIBLE_DEVICES=gpu
import numpy as np
import pandas as pd
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data.sampler import SubsetRandomSampler
import matplotlib.pyplot as plt
import time
import copy
from random import shuffle
import tqdm.notebook as tqdm
import sklearn
from sklearn.metrics import accuracy_score, cohen_kappa_score
from sklearn.metrics import classification_report
from PIL import Image
import cv2
import shutil
import seaborn as sns
import os
import copy
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torchvision
from glob import glob
from sklearn.metrics import roc_curve,auc, precision_score,precision_recall_curve,recall_score,precision_recall_fscore_support,confusion_matrix
import numpy as np
from sklearn.model_selection import train_test_split
from torchvision import models
from prettytable import PrettyTable
print(torch.cuda.is_available())
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
print(torch.cuda.get_device_properties(0).total_memory)
print(torch.cuda.memory_allocated())
gpu_id = gpu

True
11721506816
255093760


In [53]:
torch.cuda.empty_cache()
device=torch.device("cuda:3")
torch.cuda.set_device(gpu)
print("Training on GPU... Ready for HyperJump...")

Training on GPU... Ready for HyperJump...


In [54]:
def densenet_Model(pretrained=True):
    model = models.densenet121(pretrained=pretrained) # Returns Defined Densenet model with weights trained on ImageNet
    num_ftrs = model.classifier.in_features # Get the number of features output from CNN layer
    model.classifier = nn.Linear(num_ftrs, 1) # Overwrites the Classifier layer with custom defined layer for transfer learning
    model = model.to(device) # Transfer the Model to GPU if available
    return model

In [55]:
path="/DATA/chowdari1/saved_models/nih_ppr_gold(100epoch).pth"
model_nih = densenet_Model(pretrained=True)
model_nih=model_nih.to(device)
x_nih=torch.load(path,map_location='cpu')

In [56]:
path="/DATA/chowdari1/saved_models/stanford_gold(100epoch).pth"
model_stanford = densenet_Model(pretrained=True)
x_stanford=torch.load(path,map_location='cpu')

In [57]:
path="/DATA/chowdari1/saved_models/vinbig_15k(100epoch).pth"
model_vinbig = densenet_Model(pretrained=True)
x_vinbig=torch.load(path,map_location='cpu')

In [58]:
model_global = densenet_Model(pretrained=True)
x_global=torch.load(path)

In [59]:
keys=x_stanford.keys()
len(keys)
print(type(keys))
p=60
q=51
r=18
abc=0
print('start')
for i in keys:
    x_nih[i]=x_nih[i].to(device)
    a=x_nih[i].float()
    a*=(p/(p+q+r))
    x_stanford[i]=x_stanford[i].to(device)
    b=x_stanford[i].float()
    b*=(q/(q+p+r))
    x_vinbig[i]=x_vinbig[i].to(device)
    c=x_vinbig[i].float()
    c*=(r/(p+q+r))
    x_global[i]=torch.add(a,b)
    x_global[i]=torch.add(x_global[i],c)
    x_global[i]=x_global[i].float()
    if(abc==0):
        print(x_global[i][0][0][0])
        abc=1

<class 'odict_keys'>
start
tensor([0.0774, 0.1482, 0.1647, 0.1648, 0.1753, 0.1643, 0.1392],
       device='cuda:3')


In [60]:
path="/DATA/chowdari1/saved_models/ppr/three_val_loss_avg.pth"
model_global = densenet_Model(pretrained=True)
model_global.load_state_dict(x_global)
torch.save(model_global.state_dict(), path)
print('model saved as: ',path)

model saved as:  /DATA/chowdari1/saved_models/ppr/three_val_loss_avg.pth


In [61]:
!nvidia-smi

Tue Jan 25 16:04:48 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 440.82       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:04:00.0 Off |                  N/A |
| 42%   74C    P2    91W / 250W |   2471MiB / 11178MiB |     66%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  Off  | 00000000:06:00.0 Off |                  N/A |
| 20%   27C    P8     6W / 250W |     10MiB / 11178MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  G